## Dictionaries in Python

### Eigenschaften:
- nicht-linear (je nach Definition)
- dynamisch
- homogen (in den meisten Fällen, aber nicht immer)

#### Verhalten:
- Hash-basiert &rarr; `key` wird über Hashfunktion in einen Index umgewandelt an dem `value` gespeichert wird

#### Operatoren:
- Notwendige:
  - Erzeugen eines leeren Dictionaries --> wird meistens nicht explizit aufgelistet
  - `my_dict['key']` bzw. `my_dict.get('key')`: Zugriff auf Element mit Schlüssel `key` im Dictionary
  - `del my_dict['key']`: Löschen des Elements mit Schlüssel `key` im Dictionary
- Hilfreiche:
  - `items`: Gibt eine Liste von Tupeln zurück, die jeweils ein Element des Dictionaries darstellen
  - `keys`: Gibt eine Liste der Schlüssel des Dictionaries zurück
  - `values`: Gibt eine Liste der Werte des Dictionaries zurück

---
## Erzeugen eines leeren Dictionaries & befüllen mit Werten

In [1]:
my_dict = {}

my_dict['key1'] = 'value1' # add a key-value pair
my_dict[1] = 5 # the type of neither key nor value is generally restricted
my_dict['key2'] = 10

a_list = [1, 2, 3, 4, 5]
my_dict['key3'] = a_list # a value can be a list

print(my_dict)

{'key1': 'value1', 1: 5, 'key2': 10, 'key3': [1, 2, 3, 4, 5]}


Die Schlüssel müssen aber unveränderlich (immutable) sein, also z.B. `int`, `float`, `str`, `tuple`, aber nicht `list` oder `dict`.  
Nur in diesem Fall lässt sich ein eindeutiger Hashwert berechnen.

In [3]:
#my_dict[a_list] = len(a_list)
my_dict[tuple(a_list)] = len(a_list)
print(my_dict[tuple(a_list)])
a_list.append(6)
print(my_dict[tuple(a_list)])

5


KeyError: (1, 2, 3, 4, 5, 6)

### Zugriff auf Elemente

In [4]:
print(my_dict['key1']) # access a value by its key

try:
    print(my_dict['not_exist'])
except KeyError as e:
    print(F"Accessing the key {e} in a read-only-manner raises an exception")

value1
Accessing the key 'not_exist' in a read-only-manner raises an exception


### Iterieren über `keys` und `values`
Wir können auch über die `keys` und `values` unseres Dictionaries iterieren.

In [5]:
my_values = my_dict.values()

for value in my_values:
    print(F"{type(value)} -> {value}")

<class 'str'> -> value1
<class 'int'> -> 5
<class 'int'> -> 10
<class 'list'> -> [1, 2, 3, 4, 5, 6]
<class 'int'> -> 5


In [6]:
my_keys = my_dict.keys()

for key in my_keys:
    print(F"{type(key)} -> {key}")

<class 'str'> -> key1
<class 'int'> -> 1
<class 'str'> -> key2
<class 'str'> -> key3
<class 'tuple'> -> (1, 2, 3, 4, 5)


Suche nach Keys in einem Dictionary:

In [11]:
"key" in my_dict.keys()

print(my_dict.get(("key", 0)))

for key, value in my_dict.items():
    print(f"{key} - {value}")

None
key1 - value1
1 - 5
key2 - 10
key3 - [1, 2, 3, 4, 5, 6]
(1, 2, 3, 4, 5) - 5


### Hashing von eigenen Klassen
Wir können auch eigene Klassen in Dictionaries als `key` verwenden, müssen aber gewährleisten dass die Klassen vergleichbar und hashbar sind.  
Für unsere Sensor-Klasse ist dies noch nicht der Fall.

In [12]:
import time
from datetime import datetime

class Sensor:
    def __init__(self, id, sens_type, sensitivity):
        self.id = id
        self.sens_type = sens_type
        self.sensitivity = sensitivity

    def __str__(self) -> str:
        return F"Sensor: (ID: {self.id} | Type: {self.sens_type} | Sensitivity: {self.sensitivity})"
    
    def __repr__(self) -> str:
        return self.__str__()

sensor_1 = Sensor(id=1, sens_type="Temperature", sensitivity=0.5)
sensor_2 = Sensor(id=1, sens_type="Temperature", sensitivity=0.5)

sensor_deployment_dict = {}

sensor_deployment_dict[sensor_1] = datetime.now()
print(sensor_deployment_dict)

time.sleep(1) # wait for a second

# The two sensors are identical in terms of their attributes, but they are not the same object!
sensor_deployment_dict[sensor_2] = datetime.now()
print(sensor_deployment_dict)

{Sensor: (ID: 1 | Type: Temperature | Sensitivity: 0.5): datetime.datetime(2024, 12, 5, 11, 23, 46, 594259)}
{Sensor: (ID: 1 | Type: Temperature | Sensitivity: 0.5): datetime.datetime(2024, 12, 5, 11, 23, 46, 594259), Sensor: (ID: 1 | Type: Temperature | Sensitivity: 0.5): datetime.datetime(2024, 12, 5, 11, 23, 47, 594572)}


Durch die Abgeleitete Klasse `HashableSensor` können wir die Klasse `Sensor` hashbar machen.  
Hierzu müssen wir die Methoden `__eq__` und `__hash__` überschreiben.

In [13]:
class HashableSensor(Sensor):
    def __eq__(self, __value: object) -> bool:
        return self.id == __value.id and self.sens_type == __value.sens_type and self.sensitivity == __value.sensitivity
    
    def __hash__(self) -> int:
        return hash((self.id, self.sens_type, self.sensitivity))

hashable_sensor_1 = HashableSensor(1, "Temperature", 0.5)
hashable_sensor_2 = HashableSensor(1, "Temperature", 0.5)
# The two sensors are identical in terms of their attributes, and are now also identified as the same object due to the __eq__ and __hash__ methods!

sensor_deployment_dict.clear()

sensor_deployment_dict[hashable_sensor_1] = datetime.now()
print(sensor_deployment_dict)

time.sleep(1) # wait for a second

sensor_deployment_dict[hashable_sensor_2] = datetime.now()
print(sensor_deployment_dict)

{Sensor: (ID: 1 | Type: Temperature | Sensitivity: 0.5): datetime.datetime(2024, 12, 5, 11, 32, 11, 983131)}
{Sensor: (ID: 1 | Type: Temperature | Sensitivity: 0.5): datetime.datetime(2024, 12, 5, 11, 32, 12, 983429)}


### 🤓 Sortierbar-Machen von eigenen Klassen

Unabhäning von der Hashbarkeit können wir auch die Klasse auf ähnliche Art und Weise sortierbar machen. Hierzu müssen wir die Methode `__lt__` erstellen und/oder überschreiben. Python sortiert dann die Objekte anhand des Rückgabewertes dieser Methode. Python nutzt hierfür den sogenannten Timsort-Algorithmus.

In [14]:
class StortableSensor(Sensor):
    def __lt__(self, other) -> bool:
        return self.id <= other.id
    
sortableSensor_1 = StortableSensor(11, "Temperature", 0.5)	
sortableSensor_2 = StortableSensor( 2, "Temperature", 0.5)
sortableSensor_3 = StortableSensor(13, "Temperature", 0.5)

sensors = [sortableSensor_1, sortableSensor_2, sortableSensor_3]

sensors.sort()

print(sensors)

[Sensor: (ID: 2 | Type: Temperature | Sensitivity: 0.5), Sensor: (ID: 11 | Type: Temperature | Sensitivity: 0.5), Sensor: (ID: 13 | Type: Temperature | Sensitivity: 0.5)]
